# **Model Training using Annotated Datasets**

In actuality, this notebook is created after the `fine_tuning_for_RE.ipynb` google colab notebook. Hence, most of the code from that notebook has been reusd here, just for the reasons of testing the performance of both SpanBERT and RoBERTa for relation extraction tasks without training and fine-tuning. Given this, a more comprehensive outline of the source code is found in the stated notebook.

As such, some important points for base case testing is:
1. Only the `final_text.xlxs` file is required to be in the root directory of the mounted drive
2. No model saving, no training, just importing the model from HuggingFace and goes straight to testing.
3. As such, both input formatting and tokenization still follows, also preserving the max sequences of tokens with a value of 128.

In general, these trials reveal that both models perform on the downstream RE task poorly without and training on the given dataset.

## **Preliminaries**

In [ ]:
pip install pandas openpyxl

In [ ]:
#Mounting Drive for excel files and saving models
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

## **Base Case SpanBERT**

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

test_file = "/content/drive/MyDrive/final_test.xlsx"

SPANBERT_MODEL = "SpanBERT/spanbert-base-cased"

def load_test_data(file_path):
    df = pd.read_excel(file_path)
    df = df.rename(columns={
        "Chunk": "chunk",
        "Entity 1": "entity1",
        "Entity 2": "entity2",
        "Entity 1 Type": "entity_type1",
        "Entity 2 Type": "entity_type2",
        "relation": "relation"
    })
    return Dataset.from_pandas(df)

def preprocess_test_data(batch, tokenizer, model_name, max_seq_length):
    inputs = []
    for chunk, entity1, entity2, entity_type1, entity_type2 in zip(
        batch["chunk"], batch["entity1"], batch["entity2"], batch["entity_type1"], batch["entity_type2"]
    ):
        if "spanbert" in model_name:
            input_text = chunk.replace(entity1, f"[{entity_type1}]").replace(entity2, f"[{entity_type2}]")
        else:
            input_text = chunk.replace(entity1, f"[{entity_type1}]").replace(entity2, f"[{entity_type2}]")
        inputs.append(input_text)

    return tokenizer(inputs, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")

def compute_metrics_base_case(predictions, labels):
    preds = predictions.argmax(dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

test_dataset = load_test_data(test_file)

relations = test_dataset["relation"]
label_to_id = {label: idx for idx, label in enumerate(set(relations))}
id_to_label = {v: k for k, v in label_to_id.items()}
test_dataset = test_dataset.map(lambda example: {"label": label_to_id[example["relation"]]})

tokenizer = AutoTokenizer.from_pretrained(SPANBERT_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(SPANBERT_MODEL, num_labels=len(label_to_id))
model.eval()

max_seq_length = 128
test_encodings = preprocess_test_data(test_dataset, tokenizer, SPANBERT_MODEL, max_seq_length)
labels = torch.tensor(test_dataset["label"])

with torch.no_grad():    # employed to avoid RAM crashing
    outputs = model(**test_encodings)
    logits = outputs.logits

metrics = compute_metrics_base_case(logits, labels)
print(metrics)

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/215M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/215M [00:00<?, ?B/s]

{'accuracy': 0.3978779840848806, 'f1': 0.3413916032088351, 'precision': 0.3783372984403912, 'recall': 0.3314935717902186}


## **Base Case RoBERTa**

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

test_file = "/content/drive/MyDrive/final_test.xlsx"

ROBERTA_MODEL = "roberta-base"

def load_test_data(file_path):
    df = pd.read_excel(file_path)
    df = df.rename(columns={
        "Chunk": "chunk",
        "Entity 1": "entity1",
        "Entity 2": "entity2",
        "Entity 1 Type": "entity_type1",
        "Entity 2 Type": "entity_type2",
        "relation": "relation"
    })
    return Dataset.from_pandas(df)

def preprocess_test_data(batch, tokenizer, model_name, max_seq_length):
    inputs = []
    for chunk, entity1, entity2, entity_type1, entity_type2 in zip(
        batch["chunk"], batch["entity1"], batch["entity2"], batch["entity_type1"], batch["entity_type2"]
    ):
        input_text = chunk.replace(entity1, f"[{entity_type1}]").replace(entity2, f"[{entity_type2}]")
        inputs.append(input_text)

    return tokenizer(inputs, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")

def compute_metrics_base_case(predictions, labels):
    preds = predictions.argmax(dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

test_dataset = load_test_data(test_file)

relations = test_dataset["relation"]
label_to_id = {label: idx for idx, label in enumerate(set(relations))}
id_to_label = {v: k for k, v in label_to_id.items()}
test_dataset = test_dataset.map(lambda example: {"label": label_to_id[example["relation"]]})

tokenizer = AutoTokenizer.from_pretrained(ROBERTA_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(ROBERTA_MODEL, num_labels=len(label_to_id))
model.eval()

max_seq_length = 128
test_encodings = preprocess_test_data(test_dataset, tokenizer, ROBERTA_MODEL, max_seq_length)
labels = torch.tensor(test_dataset["label"])

with torch.no_grad(): #added to avoid RAM crash
    outputs = model(**test_encodings)
    logits = outputs.logits


metrics = compute_metrics_base_case(logits, labels)
print(metrics)

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.5172413793103449, 'f1': 0.3166251791506374, 'precision': 0.347313596491228, 'recall': 0.3845305086615911}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
